In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UnfoldingLSTM(nn.ModuleList):
	def __init__(self, args):
	super(UnfoldingLSTM, self).__init__()
	
	# Number of samples per time step
	self.batch_size = 2
	
	# Dimension of weight vectors
	self.hidden_dim = 16
	
	# Dimension of embedded tensor
	self.embedding_dim = 2
	
	# The vocabulary size
	self.input_size = 4
	
	# Number of time steps
	self.sequence_len = 2
	
	# Initialize embedding layer
	self.embedding = nn.Embedding(self.input_size, self.embedding_dim, padding_idx=0)
	
	# Initialize LSTM Cell
	self.lstm_cell = nn.LSTMCell(self.embedding_dim, self.hidden_dim)

In [ ]:
# Initialize LSTM Cell for the first layer
self.lstm_cell_layer_1 = nn.LSTMCell(self.embedding_dim, self.hidden_dim)

# Initialize LSTM Cell for the second layer
self.lstm_cell_layer_2 = nn.LSTMCell(self.hidden_dim, self.hidden_dim)


# Initialize LSTM Cell for the second layer
self.lstm_cell_layer_2 = nn.LSTMCell(self.hidden_dim, self.hidden_dim)

def forward(self, x):
	
	# batch_size x hidden_size
	hidden_state = torch.zeros(x.size(0), self.hidden_dim)
	cell_state = torch.zeros(x.size(0), self.hidden_dim)
	hidden_state_2 = torch.zeros(x.size(0), self.hidden_dim)
	cell_state_2 = torch.zeros(x.size(0), self.hidden_dim)
 	hidden_state_3 = torch.zeros(x.size(0), self.hidden_dim)
	cell_state_3 = torch.zeros(x.size(0), self.hidden_dim)
	
	# weights initialization
	torch.nn.init.xavier_normal_(hidden_state)
	torch.nn.init.xavier_normal_(cell_state)
	torch.nn.init.xavier_normal_(hidden_state_2)
	torch.nn.init.xavier_normal_(cell_state_2)
 	torch.nn.init.xavier_normal_(hidden_state_3)
	torch.nn.init.xavier_normal_(cell_state_3)
	
	# From idx to embedding
	out = self.embedding(x)
	
	# Prepare the shape for LSTMCell
	out = out.view(self.sequence_len, x.size(0), -1)
	
	# Unfolding LSTM
	# Last hidden_state will be used to feed the fully connected neural net
	for i in range(self.sequence_len):
		hidden_state, cell_state = self.lstm_cell_1(out[i], (hidden_state, cell_state))
	  
    if i % step_1 == 0:

		    hidden_state_2, cell_state_2 = self.lstm_cell_2(hidden_state, (hidden_state_2, cell_state_2))
			
		if i % step_1 * step_2 == 0:
			  
				hidden_state_3, cell_state_3 = self.lstm_cell_2(hidden_state, (hidden_state_3, cell_state_3))
		
	# Last hidden state is passed through a fully connected neural net
	out = self.fully_connected(hidden_state_2)	
	
	return out

In [ ]:
import math
import numpy as np

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F

import pdb

"""
Variable-length attention mechanism in hierarchical LSTM
"""

class VLAttHierLstm(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(VLAttHierLstm, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # default maximum upgrade length
        self.up_len = 80

        # bottom layer of LSTM
        self.bot_lstm = nn.LSTM(input_size, hidden_size, num_layers)             # lstm = rnn.LSTM(10, num_layers = 1, layout = 'NTC')

        # top layer of LSTM
        self.top_lstm = nn.LSTM(hidden_size, hidden_size, num_layers)            # lstm = rnn.LSTM(10, num_layers = 1, layout = 'NTC')

        # affine transformation for lstm hidden state
        self.att_hw = nn.Linear(hidden_size, hidden_size)                        # mx.symbol.FullyConnected

        # affine transformation for context
        self.att_cw = nn.Linear(hidden_size, hidden_size)

        # attention bias
        self.att_bias = nn.Parameter(torch.zeros(hidden_size))

        # affine transformation for vector to scalar
        self.att_v2s = nn.Linear(hidden_size, 1)

        # # initial states
        # self.h_0 = autograd.Variable(torch.zeros(1, 1, hidden_size)).cuda()
        # self.c_0 = autograd.Variable(torch.zeros(1, 1, hidden_size)).cuda()

    def forward(self, x):
        bot_lstm_out, _ = self.bot_lstm(x)  # output, (hidden, cell) = lstm(input_data, begin_state)

        output = None   # output of VLAttHierLstm

        seq_len = len(x)

        # set upgrade length
        up_len = min(self.up_len, math.floor(math.sqrt(seq_len)))
        # evenly spaced index
        idx = np.linspace(up_len - 1, math.pow(up_len, 2) - 1, num = up_len)
        # input for top lstm
        up_x = torch.cat([bot_lstm_out[i] for i in idx])
        # append the last output of bottom lstm
        if idx[-1] != seq_len - 1:
            up_x = torch.cat((up_x, bot_lstm_out[-1]))

        # top_h_n is ctx (context)
        _, (ctx, _) = self.top_lstm(up_x.view(len(up_x), 1, -1))

        for i in range(seq_len):
            y = bot_lstm_out[: (i + 1)]
            m = F.tanh(self.att_hw(y) + self.att_cw(ctx.expand(i + 1, -1, -1))
                       + self.att_bias.view(1, 1, -1).expand(i + 1, -1, -1))
            m = self.att_v2s(m)
            s = F.softmax(m, dim = 0)
            z = torch.sum(y * s, dim = 0).view(1, 1, -1)    # broadcasting when multiply y ans s

            if output is None:
                output = z
            else:
                output = torch.cat((output, z), dim = 0)



        return output